In [ ]:
from llm.LLMModel import *
from ner.llm_ner.prompt_techniques.pt_multi_pt import PT_Multi_PT, PT_2Time_Tagger
from ner.llm_ner.prompt_techniques.pt_discussion import PT_OutputList
from ner.Datasets.Conll2003Dataset import Conll2003Dataset
from ner.Datasets.MyDataset import MyDataset
from ner.llm_ner.few_shots_techniques import *
%reload_ext autoreload
%autoreload 2
from ner.evaluating_confidence import *

In [ ]:
data_train = MyDataset.my_load_dataset(dataset=Conll2003Dataset, split = 'train', cleaned= True)
data_test =  MyDataset.my_load_dataset(dataset=Conll2003Dataset, split = 'test', cleaned= True)

In [ ]:
model = MistralAI(llm_loader = Llama_LlamaCpp())
fst = FST_Sentence(data_train, 3)
multi_pt = PT_2Time_Tagger(fst)

In [ ]:
sentence = data_test[18]
logits_for_tags, output, index, values = get_logits_for_tags(sentence, model, pt = multi_pt)
logits_for_tags

In [ ]:
generated_tokens = list(model.model.model.eval_tokens)[-output['usage']['completion_tokens']:]
generated_logits = list(model.model.model.eval_logits)[-output['usage']['completion_tokens']:]

In [ ]:
[logits[28753] for logits in generated_logits ]

In [ ]:
logits_for_tags = [{'gold_tag' : gold_tag, 'tags_logits': {tag: generated_logits[idx][mapping_letter_tokens[tag][1]] for tag in mapping_letter_tokens.keys()}} for idx, gold_tag in zip(index,values)]
logits_for_tags

In [ ]:
# Afficher n = 0 et n = 2 où LOC est attendu mais on donne du ORG
n = 2
sentence = data_test[n]['text']
print(data_test[n]['text'])
print(data_test[n]['spans'])
get_logits_for_tags(sentence, model, pt)

In [ ]:
sentence = "Washington is the president of the United States"
sentence_o = " [['Washington', 'LOC'], ['United States', 'LOC']] "
prompt = llama_ner.get_prompts(sentence=sentence, tags = [], n = 0)

output = llm(prompt, stop = ["<end_output>"])

In [ ]:
from ner.evaluating_confidence import get_logits_for_tags
sentences = [
    "Washington is the president of the United States",
    "John is an important person living in Paris"
]
for sent in sentences :
    print(get_logits_for_tags(sent, llm, llama_ner))

In [ ]:
model.model.detokenize([464, 28753, 647])

In [ ]:
# print(model.model.tokenize("'World Cup': 'P'".encode()))'World Cup': 'P'
# print(model.model.tokenize("'World Cup': 'P'".encode()))

print(model.model.tokenize("'World Cup': 'P', ".encode()))
print(model.model.tokenize("': 'M', ".encode()))
print(model.model.tokenize(": 'L', ".encode()))
print(model.model.tokenize(": 'O', ".encode()))
print(model.model.tokenize(": 'N', ".encode()))

In [ ]:
list(model.model.eval_tokens)

In [ ]:
model.model.model.detokenize(list(model.model.model.eval_tokens))

In [ ]:
generate_data_for_confidence()


In [ ]:

data_test =  MyDataset.my_load_dataset(dataset=Conll2003Dataset, split = 'test', cleaned= True)
data_test.select(range(100))

In [ ]:

from ner.evaluating_confidence import *

%reload_ext autoreload
%autoreload 2
all_data = load_generated_data_for_confidence()

In [ ]:
points = [] # (right/false, confidence)
for data_point in all_data:
    for entity_point in data_point['logits_for_tags'] :
        points.append(
            (entity_point['gold tag'] == entity_point['outputted_tag'],
            entity_point['confidence'][entity_point['outputted_tag']])
        )
true_values =  [pair[1] for pair in points if pair[0]]
false_values = [pair[1] for pair in points if not pair[0]]

In [ ]:
text = 'Syria'
[(data_point['text'], data_point['spans']) for data_point in data_test.dataset if text in data_point['text']]

In [ ]:
[[entity for entity in data['logits_for_tags'] if entity['gold tag'] != entity['outputted_tag'] ] for data in all_data]

In [ ]:
for data_point in all_data:
    gold_tags = {key.lower() : val for key, val in dict(data_point['spans']).items()}
    for entity_point in data_point['logits_for_tags'] :
        entity = entity_point['entity'][0].lower()
        entity_point['gold tag'] = gold_tags[entity] if entity in gold_tags else 'None'

In [ ]:
for data_point in all_data:
    for entity_point in data_point['logits_for_tags'] :
        entity_point = add_confidence_to_results(entity_point)

In [ ]:
list(zip(*points))

In [ ]:
show_confidence()

In [ ]:
for dp in all_data:
    for ent in dp['logits_for_tags'] :
        ent = add_confidence_to_results(ent)